# SCRIPT DE AUMENTO DE DATOS (DATA AUGMENTATION)

Este script genera variaciones sintéticas de un conjunto de datos de imágenes y máscaras de segmentación. Aplica transformaciones geométricas y radiométricas para aumentar la diversidad del dataset y evitar el overfitting.

In [ ]:
import tensorflow as tf
import os

# === CONFIGURACIÓN DE DIRECTORIOS ===
# Rutas de entrada (imágenes originales y sus máscaras)
IMG_DIR = "images_principales"
MASK_DIR = "mask_principales"

# Rutas de salida (donde se guardarán los datos generados)
OUTPUT_IMG_DIR = "Aum_images"
OUTPUT_MASK_DIR = "Aum_masks"

# Crear directorios de salida si no existen para evitar errores
os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)

def augment(img, mask):
    """
    Aplica transformaciones aleatorias sincronizadas a una imagen y su máscara.
    
    Args:
        img (tensor): Imagen original (RGB).
        mask (tensor): Máscara binaria asociada.
        
    Returns:
        tuple: Par (imagen, máscara) transformado.
    """
    
    # -------------------------------------------------------------------------
    # TRANSFORMACIONES GEOMÉTRICAS (Deben aplicarse a Imagen Y Máscara)
    # -------------------------------------------------------------------------
    # Nota: Es crucial aplicar la misma transformación geométrica a ambas para
    # que la etiqueta (máscara) siga coincidiendo con el objeto en la imagen.
    
    # Rotación de 90 grados (Probabilidad: 50%)
    if tf.random.uniform(()) > 0.5:
        img = tf.image.rot90(img)
        mask = tf.image.rot90(mask)
    
    # Volteo horizontal / Espejo (Probabilidad: 50%)
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    
    # Volteo vertical (Probabilidad: 50%)
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_up_down(img)
        mask = tf.image.flip_up_down(mask)
    
    # -------------------------------------------------------------------------
    # TRANSFORMACIONES RADIOMÉTRICAS (Solo se aplican a la Imagen)
    # -------------------------------------------------------------------------
    # Nota: El brillo y contraste no deben cambiar los valores de clase de la 
    # máscara binaria (0 o 1), por eso solo se modifican en 'img'.

    # Ajuste aleatorio de brillo
    if tf.random.uniform(()) > 0.5:
        img = tf.image.random_brightness(img, max_delta=0.2)

    # Ajuste aleatorio de contraste
    if tf.random.uniform(()) > 0.5:
        img = tf.image.random_contrast(img, 0.7, 1.3)

    return img, mask


def save_augmented(images_path, masks_path, num_aug=3):
    """
    Lee las imágenes del disco, aplica aumento de datos y guarda los resultados.
    
    Args:
        images_path (str): Ruta al directorio de imágenes originales.
        masks_path (str): Ruta al directorio de máscaras originales.
        num_aug (int): Número de variaciones a generar por cada imagen original.
    """
    # Listar y ordenar archivos para asegurar que imagen y máscara se correspondan
    image_files = sorted(os.listdir(images_path))
    mask_files = sorted(os.listdir(masks_path))

    # Iterar sobre cada par imagen-máscara
    for img_file, mask_file in zip(image_files, mask_files):
        
        # Construir rutas completas
        img_full_path = os.path.join(images_path, img_file)
        mask_full_path = os.path.join(masks_path, mask_file)

        # --- Carga y Preprocesamiento de la Imagen ---
        img_raw = tf.io.read_file(img_full_path)
        img = tf.image.decode_png(img_raw, channels=3) # Decodificar como RGB
        img = tf.image.convert_image_dtype(img, tf.float32) # Normalizar a [0,1]

        # --- Carga y Preprocesamiento de la Máscara ---
        mask_raw = tf.io.read_file(mask_full_path)
        mask = tf.image.decode_png(mask_raw, channels=1) # Decodificar como Escala de Grises
        mask = tf.image.convert_image_dtype(mask, tf.float32)

        # --- Guardado de datos ---
        
        # 1. Guardar la imagen y máscara ORIGINALES en la carpeta de salida
        # (Esto asegura que el dataset final contenga tanto reales como sintéticas)
        tf.keras.utils.save_img(os.path.join(OUTPUT_IMG_DIR, img_file), img)
        tf.keras.utils.save_img(os.path.join(OUTPUT_MASK_DIR, mask_file), mask)

        # 2. Generar y guardar las variaciones (AUMENTO)
        for i in range(num_aug):
            # Aplicar la función de aumento
            aug_img, aug_mask = augment(img, mask)
            
            # Crear nombres nuevos con sufijo indicando el índice de aumento
            # Nota: Asume extensiones .jpg para imagen y .png para máscaras
            new_img_name = img_file.replace(".jpg", f"_au{i}.jpg")
            new_mask_name = mask_file.replace(".png", f"_au{i}.png")

            # Guardar las versiones aumentadas
            tf.keras.utils.save_img(os.path.join(OUTPUT_IMG_DIR, new_img_name), aug_img)
            tf.keras.utils.save_img(os.path.join(OUTPUT_MASK_DIR, new_mask_name), aug_mask)


# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    # Generar 5 variaciones por cada imagen original
    print("Iniciando proceso de aumento de datos...")
    save_augmented(IMG_DIR, MASK_DIR, num_aug=5) 
    print("Aumento de datos completado exitosamente.")